# DSPY Learn - Modules

https://dspy.ai/learn/programming/modules/

In [2]:
print('abc')

abc


In [3]:
#!pip install dspy

In [4]:
import dspy 
from common.my_settings import MySettings  
from common.utils import md

settings = MySettings().get()

#lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
lm_gpt4omin = dspy.LM('gpt-4o-mini', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4omin)

lm = lm_gpt4omin
lm('hello there')


Getting keys from environment variables


['Hello! How can I assist you today?']

In [5]:
sentence = "it's a charming and often affecting journey"
classify = dspy.Predict("sentence -> sentiment: bool")
response = classify(sentence=sentence)
print(response.sentiment)

True


In [6]:
question = "What's something great about the ColBERT retrieval module"
classify = dspy.ChainOfThought("question -> answer", n=5)
response = classify(question=question)
response.completions.answer

['One great aspect of the ColBERT retrieval module is its efficient late interaction approach, which enables high-quality document retrieval while maintaining speed, resulting in improved precision and recall without significant computational overhead.',
 'One great thing about the ColBERT retrieval module is its ability to efficiently combine the powerful contextual understanding of BERT with a late interaction mechanism, allowing for scalable and fast information retrieval while maintaining high accuracy.',
 "A great aspect of the ColBERT retrieval module is its ability to combine the power of BERT's contextual embeddings with a late interaction mechanism, enabling both high relevance in search results and efficient processing over large datasets.",
 'One great aspect of the ColBERT retrieval module is its ability to achieve efficient document retrieval by utilizing a late interaction mechanism, allowing it to maintain the rich contextual understanding of BERT while ensuring fast and

In [7]:
print(f"Reasoning: {response.reasoning}")
print(f"Answer: {response.answer}")

Reasoning: The ColBERT retrieval module is notable for its efficiency and effectiveness in retrieving relevant documents from large corpora. One of its great features is its use of late interaction between the query and the document representations, which allows for high-quality retrieval while maintaining speed. This method enables ColBERT to achieve high precision and recall by focusing on the most relevant parts of documents, rather than requiring full document comparisons upfront. Additionally, it balances the trade-off between accuracy and computational cost, making it suitable for large-scale applications.
Answer: One great aspect of the ColBERT retrieval module is its efficient late interaction approach, which enables high-quality document retrieval while maintaining speed, resulting in improved precision and recall without significant computational overhead.


# Math

Install Deno to run the Python code

In [8]:
import os, pathlib, shutil, subprocess

# Point to your Deno install (Codespaces default)
deno_bin = str(pathlib.Path.home() / ".deno" / "bin")

# Add to PATH for this kernel session
os.environ["PATH"] = deno_bin + ":" + os.environ.get("PATH","")

# (Optional) sanity checks
print("deno path:", deno_bin)
print("which deno:", shutil.which("deno"))
subprocess.run(["deno","--version"], check=True)


deno path: /home/codespace/.deno/bin
which deno: /home/codespace/.deno/bin/deno
deno 2.4.5 (stable, release, x86_64-unknown-linux-gnu)
v8 13.7.152.14-rusty
typescript 5.8.3


CompletedProcess(args=['deno', '--version'], returncode=0)

In [9]:
maths = dspy.ProgramOfThought("question -> answer")
result = maths(question="Whats the volume of a circle that is 3cm in diameter?")
md(result)

Prediction(  
    reasoning='The question asks for the volume of a sphere given a diameter of 3 cm. To find the volume, the radius must first be calculated by dividing the diameter by 2, which gives us 1.5 cm. Then, using the formula for the volume of a sphere, \\( V = \\frac{4}{3} \\pi r^3 \\), where \\( r \\) is the radius, we substitute the radius into the formula. The resulting calculation yields the volume of approximately 14.14 cm³, which is consistent with the output of the code.',  
    answer='The volume of the sphere with a diameter of 3 cm is approximately 14.14 cm³.'  
)

In [10]:
maths.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': 'Your input fields are:\n1. `question` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `generated_code` (str): python code that answers the question\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## generated_code ## ]]\n{generated_code}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        You will be given `question` and you will respond with `generated_code`.\n        Generating executable Python code that programmatically computes the correct `generated_code`.\n        After you\'re done with the computation and think you have the answer, make sure to provide your answer by calling the preloaded function `final_answer()`.\n        You should structure your answer in a dict object, like {"field_a": answer_a, ...}, evaluates to the correct value 

# RAG
https://dspy.ai/learn/programming/modules/#__tabbed_1_2


In [11]:
lm('Whats the name of the Castle that David Gregory inherited?')

['David Gregory inherited the historic Castle of Eilean Donan in Scotland. Would you like to know more about its history or significance?']

In [12]:
def search(query: str) -> list[str]:
    """Retreives abstracts from wikipedia"""
    # A COLBERT server is a vector database that can be used to store and retrieve embeddings.
    # The line below is fetching snippets from wikiped (2017) that is hosted on a COLBERT server. 
    result = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)

    md("**Search result**:", result[0])
    return [item['text'] for item in result]

rag = dspy.ChainOfThought("context, question -> response")
question = "Whats the name of the Castle that David Gregory inherited?"

md("**RAG with answerable question**")
md(rag(context=search(question), question=question))

md("**RAG with unanswerable question**")
# In the example below, the question is "Who is the president of Germany?" which cannot be answered from the context retrieved.
# It should give an answer such as "I dont know" or "The context does not provide this information.
md(rag(context=search(question), question="Who is the president of Germany?")) 

**RAG with answerable question**

**Search result**:{'text': 'David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.', 'pid': 3296134, 'rank': 1, 'score': 25.856355667114258, 'prob': 0.9928459586069872, 'long_text': 'David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.'}

Prediction(  
    reasoning='The context specifies that David Gregory, the Scottish physician, inherited Kinnairdy Castle in 1664. Therefore, the name of the castle he inherited is directly stated in the text.',  
    response='The name of the castle that David Gregory inherited is Kinnairdy Castle.'  
)

**RAG with unanswerable question**

**Search result**:{'text': 'David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.', 'pid': 3296134, 'rank': 1, 'score': 25.856355667114258, 'prob': 0.9928459586069872, 'long_text': 'David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.'}

Prediction(  
    reasoning="The context provided does not contain any information about the current president of Germany. It focuses on historical figures named David Gregory and Gregory Tarchaneiotes, discussing their contributions and roles in history but does not address modern political figures. To answer the question accurately, one would need up-to-date knowledge regarding Germany's political leadership, which is not included in the context provided.",  
    response='As of October 2023, the president of Germany is Frank-Walter Steinmeier.'  
)

# Classification
https://dspy.ai/learn/programming/modules/#__tabbed_1_3

In [13]:
from typing import Literal 

class Classify(dspy.Signature):
    """Classify the sentiment"""
    sentence: str = dspy.InputField()
    sentiment: Literal['positive', 'neutral', 'negative'] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
md(classify(sentence='It is raining this evening, the roads will be dangerous'))


Prediction(  
    sentiment='negative',  
    confidence=0.85  
)

# Information Extraction
https://dspy.ai/learn/programming/modules/#__tabbed_1_4

In [14]:
#text = "Apple Inc. announced its latest iPhone 14 today. The CEO, Tim Cook, highlighted its new features in a press release."
#text = "The DSPy framework aims to resolve consistency and reliability issues by prioritizing declarative, systematic programming over manual prompt writing."
text = """
The Mongol Empire was the largest contiguous empire in history. Originating in present-day Mongolia in East Asia, the empire at its height stretched from the Sea of Japan 
to Eastern Europe, extending northward into Siberia and east and southward into the Indian subcontinent, mounting invasions of Southeast Asia, and conquering the Iranian 
plateau; and reaching westward as far as the Levant and the Carpathian Mountains.
"""

extract = dspy.Predict("text -> title, headings: list[str], entities_and_metadata: list[dict[str, str]]")
md(extract(text=text))

Prediction(  
    title='The Vast Expanse of the Mongol Empire',  
    headings=['Introduction', 'Geographical Extent', 'Historical Significance', 'Cultural Impact'],  
    entities_and_metadata=[{'entity': 'Mongol Empire', 'metadata': 'Largest contiguous empire in history'}, {'entity': 'Mongolia', 'metadata': 'Origin of the Mongol Empire'}, {'entity': 'Sea of Japan', 'metadata': 'Eastern boundary of the empire at its height'}, {'entity': 'Eastern Europe', 'metadata': 'Western boundary of the empire at its height'}, {'entity': 'Siberia', 'metadata': 'Northern extent of the empire'}, {'entity': 'Indian subcontinent', 'metadata': 'Southern extent of the empire'}, {'entity': 'Southeast Asia', 'metadata': 'Region invaded by the Mongol Empire'}, {'entity': 'Iranian plateau', 'metadata': 'Conquered region'}, {'entity': 'Levant', 'metadata': 'Western boundary of the empire'}, {'entity': 'Carpathian Mountains', 'metadata': 'Western boundary of the empire'}]  
)

# Agent 
https://dspy.ai/learn/programming/modules/#__tabbed_1_5

In [15]:
def evaluate_math(expression: str) -> float:
    md("Python code: ", expression)
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str) -> str:
    md("Searching Wikipedia for:", query)
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")

md(pred.answer)

Searching Wikipedia for:David Gregory Kinnairdy castle year of birth

Python code: 9362158 / 1625

5761.328

In [16]:
md("**Answer**: ", pred.answer)
md("**Reasoning**: ", pred.reasoning)

**Answer**: 5761.328

**Reasoning**: David Gregory of Kinnairdy Castle was born on December 20, 1625. To find the answer, I divided 9362158 by 1625, which gave me the result of approximately 5761.33.

## Agent thought process
See how by looping through the trajectory we can see how the agent 
1. thinks on what to do 
1. decides on a tool to call to fetch info 
1. calls the tool 
1. observes the result of the tool

In [17]:
# Loop through the trajectory to see how the agent thought and acted

for r in pred.trajectory:
    md(f"**{r}**: ", ": ", pred.trajectory[r])

**thought_0**: : I need to find out the year of birth of David Gregory of Kinnairdy castle in order to perform the division of 9362158 by that year. I will search for this information on Wikipedia.

**tool_name_0**: : search_wikipedia

**tool_args_0**: : {'query': 'David Gregory Kinnairdy castle year of birth'}

**observation_0**: : ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.', 'David Castle (cricketer) | David James Castle (born 25 May 1972 in Launceston, Tasmania) is a former cricket player who represented the Tasmanian Tigers in one first-class match.', 'David Castle (philosopher) | David Castle (born 1967 in Edmonton, Alberta) is a Canadian philosopher and bioethicist. He is Vice-President of Research at the University of Victoria. Previously he was Professor and Chair of Innovations in the Life Sciences at University of Edinburgh, where he investigated how to get others to innovate. From 2006–2010 he served as Canada Research Chair in Science and Society at the University of Ottawa where he developed ideas leading to the creation of the Institute for Science, Society and Policy.']

**thought_1**: : I found that David Gregory was born on December 20, 1625. Now I can perform the division of 9362158 by 1625.

...

**observation_1**: : 5761.328

**thought_2**: : I have successfully computed the division of 9362158 by 1625, resulting in approximately 5761.33. Since I have all the information I need, I can now finish the task.

**tool_name_2**: : finish

**tool_args_2**: : {}

**observation_2**: : Completed.

# Compose multiple modules into a bigger problem

https://dspy.ai/learn/programming/modules/#how-do-i-compose-multiple-modules-into-a-bigger-program

In [18]:
search("Stephen Curry is the best 3 pointer shooter ever in the human history")

**Search result**:{'text': 'Stephen Curry | Wardell Stephen Curry II (born March 14, 1988) is an American professional basketball player for the Golden State Warriors of the National Basketball Association (NBA). Many players and analysts have called him the greatest shooter in NBA history. In 2014–15, Curry won the NBA Most Valuable Player Award and led the Warriors to their first championship since 1975. The following season, he became the first player in NBA history to be elected MVP by a unanimous vote and to lead the league in scoring while shooting above 50–40–90. That same year, the Warriors broke the record for the most wins in an NBA season. Curry helped the Warriors return to the NBA Finals for a third straight year in 2017, where he won his second NBA championship.', 'pid': 1490273, 'rank': 1, 'score': 21.953977584838867, 'prob': 0.7993087921660975, 'long_text': 'Stephen Curry | Wardell Stephen Curry II (born March 14, 1988) is an American professional basketball player for the Golden State Warriors of the National Basketball Association (NBA). Many players and analysts have called him the greatest shooter in NBA history. In 2014–15, Curry won the NBA Most Valuable Player Award and led the Warriors to their first championship since 1975. The following season, he became the first player in NBA history to be elected MVP by a unanimous vote and to lead the league in scoring while shooting above 50–40–90. That same year, the Warriors broke the record for the most wins in an NBA season. Curry helped the Warriors return to the NBA Finals for a third straight year in 2017, where he won his second NBA championship.'}

['Stephen Curry | Wardell Stephen Curry II (born March 14, 1988) is an American professional basketball player for the Golden State Warriors of the National Basketball Association (NBA). Many players and analysts have called him the greatest shooter in NBA history. In 2014–15, Curry won the NBA Most Valuable Player Award and led the Warriors to their first championship since 1975. The following season, he became the first player in NBA history to be elected MVP by a unanimous vote and to lead the league in scoring while shooting above 50–40–90. That same year, the Warriors broke the record for the most wins in an NBA season. Curry helped the Warriors return to the NBA Finals for a third straight year in 2017, where he won his second NBA championship.',
 'List of career achievements by Stephen Curry | This page details the records, statistics and career achievements of American professional basketball player Stephen Curry. Curry is a point guard for the Golden State Warriors of the Nati

In [ ]:


class Hop(dspy.Module):
    def __init__(self, num_docs=10, num_hops=3):
        self.num_docs = num_docs
        self.num_hops = num_hops

        self.generate_query = dspy.ChainOfThought('claim, notes -> query')
        self.append_notes = dspy.ChainOfThought('claim, notes, context -> new_notes: list[str], titles: list[str]')

    def forward(self, claim: str) -> list[str]:
        notes = []
        titles = []
        count = 1

        for _ in range(self.num_hops):
            print(count)
            count += 1
            query = self.generate_query(claim=claim, notes=notes).query
            print('a ########################################################')
            print(query)
            context = search(query)
            prediction = self.append_notes(claim=claim, notes=notes, context=context)
            print('c ########################################################')
            notes.extend(prediction.new_notes)

            titles.extend(prediction.titles)

        return dspy.Prediction(notes=notes, titles=list(set(titles)))

hop = Hop()
md(hop(claim="The capital of Australia is Sydney"))

1


a ########################################################
Is Sydney the capital of Australia?


**Search result**:{'text': 'Sydney (disambiguation) | Sydney is the state capital of New South Wales and the most populous city in Australia.', 'pid': 4303300, 'rank': 1, 'score': 27.817190170288086, 'prob': 0.5331079867666002, 'long_text': 'Sydney (disambiguation) | Sydney is the state capital of New South Wales and the most populous city in Australia.'}

b ########################################################
c ########################################################
2
a ########################################################
What is the capital city of Australia?


**Search result**:{'text': 'List of Australian capital cities | There are eight capital cities in Australia, each of which function as the seat of government for the state or territory in which they are located. Melbourne was the initial capital following the 1901 Federation of Australia. In 1927, the seat of national government was moved to the newly created city of Canberra, which continues to serve as the national capital.', 'pid': 2095748, 'rank': 1, 'score': 27.306137084960938, 'prob': 0.48867246589941216, 'long_text': 'List of Australian capital cities | There are eight capital cities in Australia, each of which function as the seat of government for the state or territory in which they are located. Melbourne was the initial capital following the 1901 Federation of Australia. In 1927, the seat of national government was moved to the newly created city of Canberra, which continues to serve as the national capital.'}

b ########################################################
c ########################################################
3
a ########################################################
What is the capital of Australia?


**Search result**:{'text': 'List of Australian capital cities | There are eight capital cities in Australia, each of which function as the seat of government for the state or territory in which they are located. Melbourne was the initial capital following the 1901 Federation of Australia. In 1927, the seat of national government was moved to the newly created city of Canberra, which continues to serve as the national capital.', 'pid': 2095748, 'rank': 1, 'score': 26.720687866210938, 'prob': 0.46507156373876585, 'long_text': 'List of Australian capital cities | There are eight capital cities in Australia, each of which function as the seat of government for the state or territory in which they are located. Melbourne was the initial capital following the 1901 Federation of Australia. In 1927, the seat of national government was moved to the newly created city of Canberra, which continues to serve as the national capital.'}

b ########################################################
c ########################################################


Prediction(  
    notes=['The capital of Australia is Canberra, not Sydney.', 'Sydney is the capital of New South Wales.', 'Sydney is the most populous city in Australia.', 'The capital of Australia is Canberra, not Sydney.', 'Sydney is the capital of New South Wales.', 'Sydney is the most populous city in Australia.', 'The capital of Australia is Canberra, not Sydney.', 'Sydney is the capital of New South Wales.', 'Sydney is the most populous city in Australia.'],  
    titles=['Facts about Sydney', 'History of Canberra', 'Sydney vs. Canberra: A Comparison', 'Correction: Capital of Australia', 'Australian Capital Cities', 'Importance of Canberra as the national capital', 'Capital of Australia', 'Sydney as a Major City', 'New South Wales Capital']  
)

In [25]:
for i in range(4):
    print(i, '  ################################################')
    lm.inspect_history(i)

0   ################################################




[2025-09-12T17:08:42.612266]

User message:

hello there


Response:

Hello! How can I assist you today?





[2025-09-12T17:08:44.046162]

System message:

Your input fields are:
1. `sentence` (str):
Your output fields are:
1. `sentiment` (bool):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## sentence ## ]]
{sentence}

[[ ## sentiment ## ]]
{sentiment}        # note: the value you produce must be True or False

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `sentence`, produce the fields `sentiment`.


User message:

[[ ## sentence ## ]]
it's a charming and often affecting journey

Respond with the corresponding output fields, starting with the field `[[ ## sentiment ## ]]` (must be formatted as a valid Python bool), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## sentiment ## ]]
True

[[ ##